<a href="https://colab.research.google.com/github/Sane18/Breast-Cancer-Detection/blob/main/Cancer-Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install anvil-uplink

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 2.0 MB/s eta 0:00:00


In [ ]:
import anvil.server

anvil.server.connect("server_WAZEGSI44TYWIE7SVYPDOZ76-UNEOFNFMDS2CZO2F")

Connecting to wss://anvil.works/uplink
Anvil websocket open
Connected to "Published" as SERVER


In [ ]:
from google.colab import files
files.upload()
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!chmod 700 ~/.kaggle

Saving kaggle.json to kaggle (1).json


In [ ]:
!kaggle datasets download -d aryashah2k/breast-ultrasound-images-dataset

 98% 192M/195M [00:10<00:00, 21.7MB/s]
100% 195M/195M [00:10<00:00, 18.8MB/s]


In [ ]:
import zipfile
zip_ref = zipfile.ZipFile('/content/breast-ultrasound-images-dataset.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [ ]:
import os
import random
import shutil

data_dir = "/content/Dataset_BUSI_with_GT"
train_dir = "/content/train"
valid_dir = "/content/valid"
test_dir = "/content/test"

if not os.path.exists(train_dir):
    os.makedirs(train_dir)
if not os.path.exists(valid_dir):
    os.makedirs(valid_dir)
if not os.path.exists(test_dir):
    os.makedirs(test_dir)

categories = os.listdir(data_dir)
for category in categories:
    category_dir = os.path.join(data_dir, category)
    images = os.listdir(category_dir)
    random.shuffle(images)

    num_train = int(len(images) * 0.7)
    num_valid = int(len(images) * 0.2)
    num_test = len(images) - num_train - num_valid

    train_images = images[:num_train]
    valid_images = images[num_train:num_train+num_valid]
    test_images = images[num_train+num_valid:]

    for img in train_images:
        src = os.path.join(category_dir, img)
        dst = os.path.join(train_dir, category, img)
        if not os.path.exists(os.path.join(train_dir, category)):
            os.makedirs(os.path.join(train_dir, category))
        shutil.copyfile(src, dst)

    for img in valid_images:
        src = os.path.join(category_dir, img)
        dst = os.path.join(valid_dir, category, img)
        if not os.path.exists(os.path.join(valid_dir, category)):
            os.makedirs(os.path.join(valid_dir, category))
        shutil.copyfile(src, dst)

    for img in test_images:
        src = os.path.join(category_dir, img)
        dst = os.path.join(test_dir, category, img)
        if not os.path.exists(os.path.join(test_dir, category)):
            os.makedirs(os.path.join(test_dir, category))
        shutil.copyfile(src, dst)


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import cv2
from tensorflow.keras import optimizers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import models
from tensorflow.keras import layers
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Flatten, Dropout, BatchNormalization, Conv2D, MaxPooling2D
from keras.applications.vgg16 import VGG16
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications import ResNet50V2
from keras.applications import resnet
from tensorflow.keras.metrics import BinaryAccuracy, Precision, Recall, AUC

In [ ]:
s=256
METRICS = [
      BinaryAccuracy(name='acc'),
      Precision(name='precision'),
      Recall(name='recall'),
      AUC(name='auc')
]

In [ ]:
main_dir="/content/"
train_data=ImageDataGenerator(rescale=1./255)
test_data=ImageDataGenerator(rescale=1./255)
valid_data=ImageDataGenerator(rescale=1./255)

train_dir=os.path.join(main_dir ,'train')
train_set = train_data.flow_from_directory(train_dir,target_size=(s,s),batch_size=32,class_mode='categorical')

test_dir=os.path.join(main_dir ,'test')
test_set = test_data.flow_from_directory(test_dir,target_size=(s,s),batch_size=32,class_mode='categorical')

valid_dir=os.path.join(main_dir ,'valid')
valid_set = test_data.flow_from_directory(valid_dir,target_size=(s,s),batch_size=32,class_mode='categorical')

In [ ]:
class_names=list(test_set.class_indices.keys())
class_names

In [ ]:
class_names=list(test_set.class_indices.keys())
plt.figure(figsize=(10, 10))
for _ in range(5):
    img, label = test_set.next()
    plt.subplot(1, 5, _+1)
    plt.tight_layout()
    plt.axis('off')
    plt.imshow(img[0], cmap='gray')
    plt.title(class_names[np.argmax(label[_])])

In [ ]:
input_shape = (s,s,3)

resent_model = ResNet50V2(include_top=False, weights='imagenet',input_shape=input_shape)

for layer in resent_model.layers:
    layer.trainable = False

model_res = models.Sequential()
model_res.add(resent_model)
model_res.add(layers.Flatten())
model_res.add(layers.Dense(256, activation='relu', name='Layer_1'))
model_res.add(layers.Dropout((0.2)))
model_res.add(layers.Dense(128, activation='relu', name='Layer_2'))
model_res.add(layers.Dense(3, activation='softmax'))
model_res.compile(loss='categorical_crossentropy',
              optimizer="adam",
              metrics=METRICS)

model_res.summary()

In [ ]:
from keras.callbacks import EarlyStopping

early_stop = EarlyStopping(monitor='val_loss', patience=25)

res = model_res.fit(train_set,
                validation_data = valid_set,
                epochs = 1,
                steps_per_epoch=len(train_set),
                validation_steps=len(valid_set),
                callbacks=[early_stop])

In [ ]:
acc = res.history['acc']
val_acc = res.history['val_acc']
loss = res.history['loss']
val_loss = res.history['val_loss']

iters = range(len(acc))

plt.plot(iters, acc, 'bo', label='Training acc')
plt.plot(iters, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(iters, loss, 'bo', label='Training loss')
plt.plot(iters, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [ ]:
predict = model_res.evaluate(valid_set)
accuracy_resnet=predict[1]

In [ ]:
y

In [ ]:
from tensorflow.keras.applications import NASNetMobile

input_shape = (s, s, 3)

nasnet_model = NASNetMobile(include_top=False, weights='imagenet', input_shape=input_shape)

for layer in nasnet_model.layers:
    layer.trainable = False

model_nasnet = models.Sequential()
model_nasnet.add(nasnet_model)
model_nasnet.add(layers.Flatten())
model_nasnet.add(layers.Dense(256, activation='relu', name='Layer_1'))
model_nasnet.add(layers.Dropout(0.2))
model_nasnet.add(layers.Dense(128, activation='relu', name='Layer_2'))
model_nasnet.add(layers.Dense(3, activation='softmax'))

model_nasnet.compile(loss='categorical_crossentropy', optimizer='adam', metrics=METRICS)

model_nasnet.summary()


In [ ]:
from keras.callbacks import EarlyStopping

early_stop = EarlyStopping(monitor='val_loss', patience=25)

res = model_nasnet.fit(train_set,
                validation_data = valid_set,
                epochs = 1,
                steps_per_epoch=len(train_set),
                validation_steps=len(valid_set),
                callbacks=[early_stop])

In [ ]:
acc = res.history['acc']
val_acc = res.history['val_acc']
loss = res.history['loss']
val_loss = res.history['val_loss']

iters = range(len(acc))

plt.plot(iters, acc, 'bo', label='Training acc')
plt.plot(iters, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(iters, loss, 'bo', label='Training loss')
plt.plot(iters, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [ ]:
from tensorflow.keras.applications import Xception

input_shape = (s, s, 3)

xception_model = Xception(include_top=False, weights='imagenet', input_shape=input_shape)

for layer in xception_model.layers:
    layer.trainable = False

model_xception = models.Sequential()
model_xception.add(xception_model)
model_xception.add(layers.Flatten())
model_xception.add(layers.Dense(256, activation='relu', name='Layer_1'))
model_xception.add(layers.Dropout(0.2))
model_xception.add(layers.Dense(128, activation='relu', name='Layer_2'))
model_xception.add(layers.Dense(3, activation='softmax'))

model_xception.compile(loss='categorical_crossentropy', optimizer='adam', metrics=METRICS)

model_xception.summary()


In [ ]:
from keras.callbacks import EarlyStopping

early_stop = EarlyStopping(monitor='val_loss', patience=25)

res = model_xception.fit(train_set,
                validation_data = valid_set,
                epochs = 1,
                steps_per_epoch=len(train_set),
                validation_steps=len(valid_set),
                callbacks=[early_stop])

In [ ]:
acc = res.history['acc']
val_acc = res.history['val_acc']
loss = res.history['loss']
val_loss = res.history['val_loss']

iters = range(len(acc))

plt.plot(iters, acc, 'bo', label='Training acc')
plt.plot(iters, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(iters, loss, 'bo', label='Training loss')
plt.plot(iters, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [ ]:
from tensorflow.keras.applications import MobileNetV2

input_shape = (s, s, 3)

mobilenet_model = MobileNetV2(include_top=False, weights='imagenet', input_shape=input_shape)

for layer in mobilenet_model.layers:
    layer.trainable = False

model_mobilenet = models.Sequential()
model_mobilenet.add(mobilenet_model)
model_mobilenet.add(layers.Flatten())
model_mobilenet.add(layers.Dense(256, activation='relu', name='Layer_1'))
model_mobilenet.add(layers.Dropout(0.2))
model_mobilenet.add(layers.Dense(128, activation='relu', name='Layer_2'))
model_mobilenet.add(layers.Dense(3, activation='softmax'))

model_mobilenet.compile(loss='categorical_crossentropy', optimizer='adam', metrics=METRICS)

model_mobilenet.summary()


In [ ]:
from keras.callbacks import EarlyStopping

early_stop = EarlyStopping(monitor='val_loss', patience=25)

res = model_mobilenet.fit(train_set,
                validation_data = valid_set,
                epochs = 1,
                steps_per_epoch=len(train_set),
                validation_steps=len(valid_set),
                callbacks=[early_stop])

In [ ]:
acc = res.history['acc']
val_acc = res.history['val_acc']
loss = res.history['loss']
val_loss = res.history['val_loss']

iters = range(len(acc))

plt.plot(iters, acc, 'bo', label='Training acc')
plt.plot(iters, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(iters, loss, 'bo', label='Training loss')
plt.plot(iters, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [ ]:
import keras.utils as image
from PIL import Image
import io

@anvil.server.callable
def predict_cancer(anvil_image):
    image_bytes = anvil_image.get_bytes()

    image_file = io.BytesIO(image_bytes)

    test_image = Image.open(image_file)

    resized_image = test_image.resize((256, 256))

    classes_dir = ["Benign", "Malignant", "Normal"]
    norm_img = image.img_to_array(resized_image) / 255
    input_arr_img = np.array([norm_img])
    pred = np.argmax(model_res.predict(input_arr_img))
    return classes_dir[pred]


In [ ]:
anvil.server.wait_forever()

In [ ]:
import keras.utils as image
import matplotlib.pyplot as plt

path = "/content/test/benign/benign (108)_mask.png"
classes_dir = ["Benign", "Malignant", "Normal"]
img = image.load_img(path, target_size=(s, s))
norm_img = image.img_to_array(img) / 255
input_arr_img = np.array([norm_img])
pred = np.argmax(model_res.predict(input_arr_img))

# Display the image
plt.imshow(img)
plt.axis('off')
plt.title(classes_dir[pred])
plt.show()

In [ ]:
import keras.utils as image

path = "/content/test/benign/benign (108)_mask.png"
classes_dir = ["Benign","Malignant","Normal"]
img = image.load_img(path, target_size=(s,s))
norm_img = image.img_to_array(img)/255
input_arr_img = np.array([norm_img])
pred = np.argmax(model_res.predict(input_arr_img))
print(classes_dir[pred])

In [ ]:
model_nasnet.save('/content/Model')

In [ ]:
model_nasnet.save('/content/Model.h5')